In [2]:
import os
import requests
from dotenv import load_dotenv
from openpyxl import Workbook

# Carregar variáveis de ambiente
load_dotenv()

# Obter as variáveis de ambiente
eloqua_username = os.getenv("ELOQUA_USERNAME_LALEY")
eloqua_password = os.getenv("ELOQUA_PASSWORD_LALEY")

# Substitua pelo seu URL base da Eloqua
base_url = "https://secure.p03.eloqua.com"

url = f"{base_url}/api/bulk/2.0/syncs/8244334/data"
offset = 0
limit = 1000
total_results = float('inf')
all_items = []

while offset < total_results:
    params = {
        "limit": limit,
        "offset": offset
    }

    # Adicionar autenticação ao cabeçalho da solicitação
    auth = (eloqua_username, eloqua_password)

    response = requests.get(url, params=params, auth=auth)

    if response.status_code == 200:
        data = response.json()
        total_results = data["totalResults"]
        offset += limit
        all_items.extend(data["items"])
    else:
        print(f"Erro na solicitação. Código de status: {response.status_code}")
        break

# Função para salvar os dados em um arquivo XLSX


def save_to_excel(data, filename):
    wb = Workbook()
    ws = wb.active

    # Escrever cabeçalhos
    csv_columns = data[0].keys()
    for i, col_name in enumerate(csv_columns):
        ws.cell(row=1, column=i+1).value = col_name

    # Escrever dados
    for row_num, item in enumerate(data, start=2):
        for col_num, value in enumerate(item.values()):
            ws.cell(row=row_num, column=col_num+1).value = value

    # Salvar arquivo
    wb.save(filename)

    print(f"Logs salvos com sucesso em {filename}!")


# Definir o limite máximo de linhas por arquivo
max_rows_per_file = 1000000

# Dividir os dados em lotes para salvar em vários arquivos
for i in range(0, len(all_items), max_rows_per_file):
    data = all_items[i:i+max_rows_per_file]
    file_number = i // max_rows_per_file + 1
    filename = f"eloqua_2023_EmailOpen_{file_number}.xlsx"
    save_to_excel(data, filename)

Logs salvos com sucesso em eloqua_2023_EmailOpen_1.xlsx!
Logs salvos com sucesso em eloqua_2023_EmailOpen_2.xlsx!
Logs salvos com sucesso em eloqua_2023_EmailOpen_3.xlsx!
Logs salvos com sucesso em eloqua_2023_EmailOpen_4.xlsx!
Logs salvos com sucesso em eloqua_2023_EmailOpen_5.xlsx!
